In [ ]:
import numpy as np
import torch
from datasets import load_dataset, Features, Sequence, Value
from sklearn.metrics import precision_recall_fscore_support
from transformers import BartTokenizer, BartForSequenceClassification, TrainingArguments, Trainer

dataset = load_dataset("go_emotions")

model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(
    model_name,
    num_labels=28,
    problem_type="multi_label_classification"
)
print(dataset)
print(model)

In [ ]:
# Tokenize and one-hot encode labels
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=128
    )

    # One-hot encode the labels
    num_classes = 28
    one_hot_labels = np.zeros((len(examples["labels"]), num_classes), dtype=np.float32)
    for i, labels in enumerate(examples["labels"]):
        one_hot_labels[i, labels] = 1.0

    tokenized["labels"] = one_hot_labels
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
new_features = Features(
    {
        "text": Value("string"),
        "labels": Sequence(Value("float32")),
        "id": Value("string"),
        "input_ids": Sequence(Value("int32")),
        "attention_mask": Sequence(Value("int32")),
    }
)

# Cast the dataset to the new type
tokenized_dataset = tokenized_dataset.cast(new_features)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=False,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="micro"
    )
    return {"precision": precision, "recall": recall, "f1": f1}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(100)),
    eval_dataset=tokenized_dataset["validation"].select(range(100)),
    tokenizer=tokenizer,
)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Evaluate the model on the test set
results = trainer.evaluate(tokenized_dataset["test"])
print("Test Results:", results)

# Save the fine-tuned model
trainer.save_model("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")